# OpenAI Function Calling 에이전트 예제

이 노트북은 OpenAI의 Function Calling 기능을 사용하여 AI 에이전트를 구현합니다.

## 1. 라이브러리 임포트

In [7]:
import os, json
from dotenv import load_dotenv
import openai
from openai.types.chat import ChatCompletionMessage

## 2. Tool 함수 정의

AI가 호출할 수 있는 함수들을 정의합니다.

In [8]:
def get_weather(city):
    """특정 도시의 날씨 정보를 반환합니다."""
    return f"The weather in {city} is sunny with a high of 100°C."

## 3. AI 응답 처리 함수

AI의 응답을 처리하고 tool_calls가 있으면 함수를 실행합니다.

In [9]:
def process_ai_response(message: ChatCompletionMessage):
    """AI 응답을 처리합니다. tool_calls가 있으면 함수를 실행하고 재귀 호출합니다."""
    
    if message.tool_calls:
        # AI가 tool을 호출한 경우: assistant 메시지를 기록
        messages.append({
            "role": "assistant",
            "content": message.content or "",
            "tool_calls": [
                {
                    "id": tool_call.id,
                    "type": "function",
                    "function": {
                        "name": tool_call.function.name,
                        "arguments": tool_call.function.arguments,
                    }
                } for tool_call in message.tool_calls
            ]
        })

        # 각 tool_call을 순회하며 실행
        for tool_call in message.tool_calls:
            function_name = tool_call.function.name
            arguments = tool_call.function.arguments

            print(f"Calling function: {function_name} with arguments: {arguments}")

            # JSON 파싱
            try:
                arguments = json.loads(arguments)
            except json.JSONDecodeError:
                arguments = {}

            # 함수 매핑에서 실제 함수 가져오기
            function_to_run = FUNCTION_MAPPINGS.get(function_name)

            # 함수 실행
            result = function_to_run(**arguments)

            # Tool 실행 결과를 메시지에 추가
            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": result
            })
        
        # Tool 실행 결과를 포함하여 AI에게 다시 요청
        call_ai()
    else:
        # Tool 호출이 없는 경우: 최종 응답
        messages.append({"role": "assistant", "content": message.content})
        print(f"AI : {message.content}")

In [10]:
def call_ai():
    """OpenAI API를 호출하여 응답을 받습니다."""
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        tools=TOOLS,
        messages=messages
    )
    
    process_ai_response(response.choices[0].message)

## 4. Tool 정의 및 매핑

OpenAI에게 사용 가능한 tool들을 알려주고, 함수 이름과 실제 함수를 매핑합니다.

In [11]:
# OpenAI API에 전달할 Tool 스키마 정의
TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get the weather information for a given city.",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "The name of the city to get the weather for."
                    }
                },
                "required": ["city"]
            }
        }
    }
]

# 함수 이름과 실제 함수를 매핑
FUNCTION_MAPPINGS = {
    "get_weather": get_weather
}

## 5. 초기화

환경 변수를 로드하고 OpenAI 클라이언트와 메시지 리스트를 초기화합니다.

In [12]:
# .env 파일에서 환경 변수 로드 (OPENAI_API_KEY)
load_dotenv()

# OpenAI 클라이언트 초기화
client = openai.OpenAI()

# 대화 히스토리를 저장할 리스트
messages = []

print("초기화 완료! 이제 대화를 시작할 수 있습니다.")

초기화 완료! 이제 대화를 시작할 수 있습니다.


## 6. 대화 실행

사용자 입력을 받아 AI와 대화합니다. 'exit' 또는 'quit'을 입력하면 종료됩니다.

In [13]:
while True:
    message = input("Send a message to the LLM...")

    if message.lower() in ["exit", "quit"]:
        print("대화를 종료합니다.")
        break
    else:
        # 사용자 메시지를 히스토리에 추가
        messages.append({"role": "user", "content": message})
        print("User :", message)
        
        # AI 호출
        call_ai()

User : 안녕 내 이름은 Origogi야
AI : 안녕하세요, Origogi님! 어떻게 도와드릴까요?
대화를 종료합니다.
